In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import time
matplotlib.use('Agg')
# 画像ディレクトリのパス
image_dir = 'data/images/'

In [ ]:
# ここで既存のモデルを読み込む
model_file = 'linear.h5'

model = tf.keras.models.load_model(model_file)
# 評価モードに設定
model.trainable = False
model.summary()
number_of_layers = len(model.layers)

def load_img(path):
    # テスト用の画像を読み込む
    img = tf.keras.preprocessing.image.load_img(path)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
# 一度だけ全ての中間層の出力を取得するモデルを作成
all_layer_outputs = [layer.output for layer in model.layers if not isinstance(layer, tf.keras.layers.Dropout)]
activation_model = Model(inputs=model.input, outputs=all_layer_outputs)

In [ ]:
def get_all_cnn_activations(img_array):
    # 全ての中間層の出力を一度に取得
    # ただし、結果として(活性化, レイヤー名)のペアをリストに格納
    activations_and_names = []
    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.Dropout):
            layer_output = layer.output
            activation_model = Model(inputs=model.input, outputs=layer_output)
            activations = activation_model.predict(img_array)
            activations_and_names.append((activations, layer.name))
    return activations_and_names

def get_all_activations(img_array):
    # 全ての中間層の出力を一度に取得
    activations_list = activation_model.predict(img_array)
    # 各層の名前を取得
    layer_names = [layer.name for layer in model.layers if not isinstance(layer, tf.keras.layers.Dropout)]
    # (活性化, レイヤー名)のペアをリストに格納
    activations_and_names = list(zip(activations_list, layer_names))
    return activations_and_names

def get_activated_image(img_array, file_id):
    # 元の画像を描画用に準備
    original_img = img_array[0].astype(np.uint8)
    # 描画するレイヤーの数をカウント
    all_activations_and_names = get_all_activations(img_array)
    count = sum(1 for (activations, layer_name) in all_activations_and_names if len(activations.shape) == 4)
    rows = count + 1  # +1 はオリジナル画像のため
    current_row = 1  # 現在の行数
    # 画像のリストを初期化
    images_to_combine = []
    # 各フィルタの活性化を表示する
    current_row = 1
    # 描画の準備
    fig = plt.figure(figsize=(20, rows), facecolor='white')  # 背景色を白に設定

    for activations, layer_name in all_activations_and_names:
        if len(activations.shape) == 4:
            n_filters = activations.shape[-1]
            combined_activation = activations[0, :, :, 0]

            side = int(np.ceil(np.sqrt(n_filters)))

            # オリジナルの画像サイズにリサイズするための準備
            resized_activations = []

            # 各フィルタの活性化をオリジナルの画像サイズにリサイズ
            for j in range(n_filters):
                resized_activation = cv2.resize(activations[0, :, :, j], (original_img.shape[1], original_img.shape[0]))
                resized_activations.append(resized_activation)

            # リサイズされた活性化を平均化
            combined_activation = np.mean(resized_activations, axis=0)
            heatmap = (combined_activation - np.min(combined_activation)) / (np.max(combined_activation) - np.min(combined_activation))

            heatmap_img = cv2.applyColorMap((heatmap * 255).astype(np.uint8), cv2.COLORMAP_JET)
            heatmap_img = cv2.cvtColor(heatmap_img, cv2.COLOR_BGR2RGB)
            overlay_img = heatmap_img
            #overlay_img  = cv2.addWeighted(original_img, 0.6, heatmap_img, 0.4, 0)
            
            ax = fig.add_subplot(rows, 1, current_row)
            ax.imshow(overlay_img)
            ax.axis('off')
            ax.set_facecolor('white')  # サブプロットの背景色を白に設定
        
            # レイヤー名を画像の左端にサブプロットのタイトルとして追加
            ax.set_title(f"Layer - {layer_name}", loc='left')
            
            current_row += 1

    # ループの後にオリジナル画像を追加
    ax = fig.add_subplot(rows, 1, current_row)
    ax.imshow(original_img)
    ax.axis('off')
    ax.set_facecolor('white')  # サブプロットの背景色を白に設定
    ax.set_title(f"Original Image: {file_id}", loc='left')
    fig.tight_layout()
    # 描画内容をnumpy配列に変換
    canvas = fig.canvas
    canvas.draw()
    img_arr = np.array(canvas.renderer.buffer_rgba())[:, :, :3]  # RGBAからRGBに変換
    plt.close(fig)
    
    return img_arr


In [ ]:
import imageio_ffmpeg as ffmpeg

# ファイル名からレコードIDを抽出する関数
def get_record_id_from_filename(filename):
    return int(filename.split('_')[0])

# 画像ディレクトリ内の全ファイルのリストを取得
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# レコードIDに基づいてファイル名をソート
image_files_sorted = sorted(image_files, key=get_record_id_from_filename)

start = 0
end = 500
count = 0
t1_total = 0
t2_total = 0
start_time = time.time()
writer_type = "ffmpeg"
#writer_type = "opencv"
# 各画像ファイルに対して処理を行う
files_to_process = image_files_sorted[start:end]
try:
    for image_file in tqdm(files_to_process, total=len(files_to_process), desc="Processing Images"):
        t1 = time.time()
        img_path = os.path.join(image_dir, image_file)
        img_array = load_img(img_path)
        file_id = get_record_id_from_filename(image_file)
        img_rgb = get_activated_image(img_array, file_id)
        t1_total += time.time() - t1
        if count == 0:
            frame_height, frame_width, _ = img_rgb.shape
            figsize = (frame_width / 100, frame_height / 100)
            default_dpi = plt.rcParams['figure.dpi']
            plt.figure(figsize=figsize, dpi=default_dpi)
            # OpenCVの画像をmatplotlibで表示
            plt.imshow(img_rgb)
            plt.axis('off')  # 軸を非表示にする
            plt.show()
            plt.close()

            # 出力動画の設定
            fps = 30.0  # 動画のフレームレート。必要に応じて変更してください。
            movie_file =  os.path.splitext(model_file)[0] + '.mp4'
            if writer_type == "ffmpeg":
                writer = ffmpeg.write_frames(movie_file, (frame_width, frame_height), codec='libx264', fps=fps)
                writer.send(None)  # Seed the writer
            elif writer_type == "opencv":
                # 出力動画の設定
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4用のコーデック
                writer = cv2.VideoWriter(movie_file, fourcc, fps, (frame_width, frame_height))

        t2 = time.time()
        # 動画ファイルを作成する
        if writer_type == "ffmpeg":
            img_rgb_contiguous = np.ascontiguousarray(img_rgb)
            writer.send(img_rgb_contiguous)
        elif writer_type == "opencv":
            img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
            writer.write(img_bgr)
        t2_total += time.time() - t2
        count += 1

finally:
    # 動画の書き込みを終了
    if writer_type == "ffmpeg":
        writer.close()
    elif writer_type == "opencv":
        writer.release()

print(f"end: {time.time() - start_time:.2f} sec, t1: {t1_total} sec, t2: {t2_total} sec")